In [1]:
!pip3 install python-dotenv

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


In [2]:
!pip3 install -q langchain langgraph duckduckgo-search torch tranformers[sentencepiece] faiss-cpu 

ERROR: Could not find a version that satisfies the requirement tranformers[sentencepiece] (from versions: none)
ERROR: No matching distribution found for tranformers[sentencepiece]

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


In [3]:
!pip3 install -q arxiv pymupdf


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
HUGGINGFACE_API_TOKEN=os.environ.get("HUGGINGFACE_API_TOKEN")

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import ArxivLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.llms.huggingface_endpoint import HuggingFaceEndpoint


In [6]:
docs = ArxivLoader(query="Retrieval Augmented Generation",load_max_docs=5).load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=350,
    chunk_overlap=50
)
chunked_documents = text_splitter.split_documents(docs)
faiss_vectorstore = FAISS.from_documents(
    documents=chunked_documents,
    embedding=FastEmbedEmbeddings()
)

/home/debian/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
special_tokens_map.json: 100%|█████████████████| 695/695 [00:00<00:00, 2.66MB/s]

tokenizer_config.json: 100%|███████████████| 1.24k/1.24k [00:00<00:00, 4.96MB/s]

Fetching 5 files:  20%|█████▍                     | 1/5 [00:01<00:06,  1.64s/it]
tokenizer.json:   0%|                                | 0.00/711k [00:00<?, ?B/s]

model_optimized.onnx:   0%|                         | 0.00/66.5M [00:00<?, ?B/s]
tokenizer.json: 100%|█████████████████████████| 711k/711k [00:00<00:00, 763kB/s]


model_optimized.onnx:  16%|██▌             | 10.5M/66.5M [00:02<00:10, 5.12MB/s]

model_optimized.onnx:  32%|█████           | 21.0M/66.5M [00:04<00:09, 5.05MB/s]

model_optimized.onnx:  47%|███████▌        | 31.5M/66.5M [00:06<00:06, 5.03MB/s]

model_opt

In [8]:
embeddings = FastEmbedEmbeddings()

In [10]:
retriever = faiss_vectorstore.as_retriever()

In [11]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [16]:
llm = HuggingFaceEndpoint(
    endpoint_url="https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2",
    task="text_generation",
    max_new_token=6096,
    huggingfacehub_api_token=HUGGINGFACE_API_TOKEN
)

WARNING! max_new_token is not default parameter.
                    max_new_token was transferred to model_kwargs.
                    Please make sure that max_new_token is what you intended.


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/debian/.cache/huggingface/token
Login successful


In [36]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}| RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | llm, "context": itemgetter("context")}
)

In [38]:
question = "What is RAG?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"])


Answer: RAG, which stands for Retrieval Augmented Generation, is a method used to complement the capabilities of large language models (LLMs) by retrieving and utilizing relevant external knowledge. It consists of three primary components: Tool Retrieval, Plan Generation, and Execution. The goal is to improve the accuracy and performance of LLMs by providing them with the right tools and context for the task at hand.
